In [ ]:
# ========== CONFIGURATION ==========
MODEL_VERSION = "single"  # Options: "single" or "two_step"

TRAIN_PATH = '/home/stargix/Desktop/hackathons/datathon/train/train'
TEST_PATH = '/home/stargix/Desktop/hackathons/datathon/test/test'
TARGET_COL = "iap_revenue_d7"
TRAIN_SAMPLE_FRAC = 0.05  # Adjust for more/less data

print(f"Selected model: {MODEL_VERSION}")

Selected model: single


In [ ]:
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
import gc
import os
from glob import glob

dask.config.set({"dataframe.convert-string": False})

: 

## Load Data (with date filters)

In [ ]:
# Train: Oct 1-5, Valid: Oct 6
filters_train = [("datetime", ">=", "2025-10-01-00-00"),
                 ("datetime", "<",  "2025-10-06-00-00")]
filters_valid = [("datetime", ">=", "2025-10-06-00-00"),
                 ("datetime", "<",  "2025-10-07-00-00")]

# Obtener lista de archivos parquet
parquet_files_all = glob(os.path.join(TRAIN_PATH, '**/part-*.parquet'), recursive=True)

# Usar un porcentaje de archivos
num_files_train = max(1, int(len(parquet_files_all) * 0.15))
parquet_files_train = parquet_files_all[:num_files_train]

print(f"Using {num_files_train} out of {len(parquet_files_all)} train files")

# Cargar TRAIN
print("Loading train data...")
dd_train = dd.read_parquet(
    parquet_files_train, 
    filters=filters_train,
    engine='pyarrow'
)

# Sample y compute
train_df = dd_train.sample(frac=TRAIN_SAMPLE_FRAC, random_state=42).compute()
print(f"Train loaded: {train_df.shape}")

# Cargar VALID
print("\nLoading validation data...")
dd_valid = dd.read_parquet(
    parquet_files_train,
    filters=filters_valid,
    engine='pyarrow'
)

valid_df = dd_valid.sample(frac=min(0.5, TRAIN_SAMPLE_FRAC), random_state=42).compute()
print(f"Valid loaded: {valid_df.shape}")

# Limpiar memoria
del dd_train, dd_valid
gc.collect()

print(f"\n✓ Data loaded successfully")

Using 21 out of 144 train files
Loading train data...


## Prepare Features (grok.ipynb approach)

In [ ]:
# Definir labels a excluir
labels_to_exclude = ['buyer_d1', 'buyer_d7', 'buyer_d14', 'buyer_d28', 'buy_d7', 'buy_d14', 'buy_d28',
                     'iap_revenue_d7', 'iap_revenue_d14', 'iap_revenue_d28', 'registration',
                     'retention_d1_to_d7', 'retention_d3_to_d7', 'retention_d7_to_d14',
                     'retention_d1', 'retention_d3', 'retention_d7']

# Definir features
features = [col for col in train_df.columns if col not in labels_to_exclude + ['row_id', 'datetime']]

# Categorical features
cat_features = ['advertiser_bundle', 'advertiser_category', 'advertiser_subcategory',
                'advertiser_bottom_taxonomy_level', 'carrier', 'country', 'region',
                'dev_make', 'dev_model', 'dev_os', 'dev_osv', 'weekday']

# Solo usar cat_features que existan
cat_features_valid = [col for col in cat_features if col in features]

print(f"Total features: {len(features)}")
print(f"Categorical features: {len(cat_features_valid)}")

# Extraer X, y
X_train = train_df[features].copy()
y_train = train_df[TARGET_COL].values

X_val = valid_df[features].copy()
y_val = valid_df[TARGET_COL].values

print(f"Train samples: {len(X_train)}")
print(f"Val samples: {len(X_val)}")

In [ ]:
# Convertir columnas object a numéricas (excepto categorical features)
numeric_cols = X_train.select_dtypes(include=['object']).columns

for col in numeric_cols:
    if col not in cat_features_valid:
        try:
            X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
            X_val[col] = pd.to_numeric(X_val[col], errors='coerce')
        except:
            pass

# Rellenar NaN generados por conversión fallida
X_train = X_train.fillna(0)
X_val = X_val.fillna(0)

# Asegurar que cat_features estén como category
for col in cat_features_valid:
    if col in X_train.columns:
        X_train[col] = X_train[col].astype('category')
        X_val[col] = X_val[col].astype('category')

print("✓ Data types converted")
print(f"Numeric features: {len([c for c in X_train.columns if c not in cat_features_valid])}")
print(f"Categorical features: {len(cat_features_valid)}")

## Train LightGBM Model (grok.ipynb approach)

In [ ]:
# Transform target to log space
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# Crear datasets LGBM
train_ds = lgb.Dataset(X_train, label=y_train_log, categorical_feature=cat_features_valid)
val_ds = lgb.Dataset(X_val, label=y_val_log, reference=train_ds)

print("✓ LightGBM datasets created")

In [ ]:
# Parámetros LGBM
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'device': 'cpu'
}

print("Parameters configured:")
for k, v in params.items():
    print(f"  {k}: {v}")

In [ ]:
# Entrenar modelo
print("Training model...")
model = lgb.train(
    params,
    train_ds,
    num_boost_round=2000,
    valid_sets=[train_ds, val_ds],
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)

print("✓ Model trained successfully")

## Evaluate

In [ ]:
# Evaluar en validación
pred_log = model.predict(X_val)
pred_val = np.expm1(pred_log).clip(0, None)
msle = mean_squared_log_error(y_val, pred_val)

msle_baseline = mean_squared_log_error(y_val, np.zeros_like(y_val))

print("=" * 50)
print("VALIDATION RESULTS")
print("=" * 50)
print(f"MSLE: {msle:.6f}")
print(f"Baseline (all zeros): {msle_baseline:.6f}")
print(f"Improvement: {((msle_baseline - msle) / msle_baseline * 100):.2f}%")

print(f"\nPrediction stats:")
print(f"  Mean: {pred_val.mean():.4f}")
print(f"  Median: {np.median(pred_val):.4f}")
print(f"  Max: {pred_val.max():.4f}")
print(f"  % Non-zero: {(pred_val > 0).mean() * 100:.2f}%")

In [ ]:
# Feature importance
lgb.plot_importance(model, max_num_features=20, figsize=(10, 8))
print("\nTop 20 most important features shown above")

In [ ]:
from glob import glob
import os

# Train: Oct 1-5, Valid: Oct 6
filters_train = [("datetime", ">=", "2025-10-01-00-00"),
                 ("datetime", "<",  "2025-10-06-00-00")]
filters_valid = [("datetime", ">=", "2025-10-06-00-00"),
                 ("datetime", "<",  "2025-10-07-00-00")]

# Obtener lista de archivos parquet
parquet_files_all = glob(os.path.join(TRAIN_PATH, '**/part-*.parquet'), recursive=True)

# 🔥 REDUCIR MÁS: Solo 5-10% de archivos
num_files_train = max(1, int(len(parquet_files_all) * 0.15))  # Cambié a 5%
parquet_files_train = parquet_files_all[:num_files_train]

print(f"Using {num_files_train} out of {len(parquet_files_all)} train files")

# 🔥 DROPEAR COLUMNAS ANTES DE COMPUTE
cols_to_drop_early = IGNORE_BIG_COLS + ["row_id", "datetime"]

# Cargar TRAIN
print("Loading train data...")
dd_train = dd.read_parquet(
    parquet_files_train, 
    filters=filters_train,
    engine='pyarrow'
)

# Dropear columnas pesadas ANTES de compute
existing_cols = [c for c in cols_to_drop_early if c in dd_train.columns]
dd_train = dd_train.drop(columns=existing_cols)

# 🔥 SAMPLE EN DASK (no en pandas)
train_sample = dd_train.sample(frac=TRAIN_SAMPLE_FRAC, random_state=42).compute()
train_sample = reduce_memory(train_sample)

print(f"Train loaded: {train_sample.shape}, Memory: {train_sample.memory_usage(deep=True).sum() / 1e9:.2f} GB")

# Limpiar memoria
del dd_train
gc.collect()

# Cargar VALID (después de liberar train)
print("\nLoading validation data...")
dd_valid = dd.read_parquet(
    parquet_files_train,  # Mismos archivos
    filters=filters_valid,
    engine='pyarrow'
)

existing_cols = [c for c in cols_to_drop_early if c in dd_valid.columns]
dd_valid = dd_valid.drop(columns=existing_cols)

# 🔥 SAMPLE MENOS EN VALID (solo necesitas evaluar, no entrenar)
valid_df = dd_valid.sample(frac=min(0.5, TRAIN_SAMPLE_FRAC), random_state=42).compute()
valid_df = reduce_memory(valid_df)

print(f"Valid loaded: {valid_df.shape}, Memory: {valid_df.memory_usage(deep=True).sum() / 1e9:.2f} GB")

del dd_valid
gc.collect()

print(f"\n✓ Data loaded successfully")
print(f"Total memory: ~{(train_sample.memory_usage(deep=True).sum() + valid_df.memory_usage(deep=True).sum()) / 1e9:.2f} GB")

Using 21 out of 144 train files
Loading train data...
Train loaded: (271487, 56), Memory: 0.40 GB
Train loaded: (271487, 56), Memory: 0.40 GB

Loading validation data...

Loading validation data...
Valid loaded: (28373, 56), Memory: 0.04 GB
Valid loaded: (28373, 56), Memory: 0.04 GB

✓ Data loaded successfully

✓ Data loaded successfully
Total memory: ~0.44 GB
Total memory: ~0.44 GB


## Generate Submission

In [ ]:
print("Generating test predictions...")

dd_test = dd.read_parquet(TEST_PATH, engine='pyarrow')
delayed_parts = dd_test.to_delayed()

print(f"Processing {len(delayed_parts)} test chunks...")

pred_dfs = []

for i, d in enumerate(delayed_parts):
    if (i + 1) % 10 == 0:
        print(f"  Chunk {i+1}/{len(delayed_parts)}...")
    
    part_df = d.compute()
    row_ids = part_df["row_id"].values
    
    # Obtener solo las features que tenemos
    X_part = part_df[[c for c in features if c in part_df.columns]].copy()
    
    # Añadir columnas faltantes con valores por defecto
    for col in features:
        if col not in X_part.columns:
            X_part[col] = 0
    
    # Reordenar columnas
    X_part = X_part[features]
    
    # Preprocessing similar al train
    numeric_cols = X_part.select_dtypes(include=['object']).columns
    for col in numeric_cols:
        if col not in cat_features_valid:
            try:
                X_part[col] = pd.to_numeric(X_part[col], errors='coerce')
            except:
                pass
    
    X_part = X_part.fillna(0)
    
    for col in cat_features_valid:
        if col in X_part.columns:
            X_part[col] = X_part[col].astype('category')
    
    # Predecir
    part_pred_log = model.predict(X_part)
    part_pred = np.expm1(part_pred_log).clip(0, None)
    
    pred_dfs.append(pd.DataFrame({
        "row_id": row_ids,
        "iap_revenue_d7": part_pred
    }))
    
    del part_df, X_part, row_ids, part_pred
    gc.collect()

# Combine and save
submission = pd.concat(pred_dfs, ignore_index=True)
output_file = "submission_grok.csv"
submission.to_csv(output_file, index=False)

print(f"\n✓ Submission saved: {output_file}")
print(f"Shape: {submission.shape}")
print(f"Sample:\n{submission.head()}")

# Validation checks
print(f"\nValidation checks:")
print(f"  NaN values: {submission.isna().sum().sum()}")
print(f"  Negative values: {(submission['iap_revenue_d7'] < 0).sum()}")

Generating test predictions...
Processing 96 test chunks...
  Chunk 10/96...
  Chunk 10/96...
  Chunk 20/96...
  Chunk 20/96...


KeyboardInterrupt: 

## Summary

This notebook uses the **grok.ipynb approach**:
- LightGBM native API (`lgb.train`) instead of sklearn wrapper
- Log-transform target (`np.log1p`)
- Explicit categorical feature handling
- Early stopping with 100 rounds
- Date-based train/validation split (Oct 1-5 for train, Oct 6 for validation)

**To adjust data size**: Change `TRAIN_SAMPLE_FRAC` in the first cell (default 0.10 = 10%)

Then re-run all cells!